# Events Module

All logic for the `/events` route, event filtering, ZIP search, and event queries migrated from `app.py`.

In [ ]:
# --- /events route and helpers ---
from flask import request, render_template
from contextlib import closing
import re

# Import get_db from the db module
import sys
from db import get_db

def get_events_page(request):
    """
    Main function called by app.py for the /events route
    """
    search_query = request.args.get('q', '').strip()
    filter_type = request.args.get('filter', 'all')
    zip_param = (request.args.get('zip', '') or '').strip()
    
    with closing(get_db()) as db:
        cur = db.cursor()
        base_query = """
            SELECT e.id, e.title, e.description, e.date, e.zip, p.business_name, p.first_name
            FROM events e
            LEFT JOIN providers p ON e.provider_id = p.id
            WHERE 1=1
        """
        params = []
        
        if search_query:
            base_query += " AND (LOWER(e.title) LIKE LOWER(?) OR LOWER(e.description) LIKE LOWER(?) OR LOWER(p.business_name) LIKE LOWER(?) OR LOWER(p.first_name) LIKE LOWER(?))"
            search_param = f"%{search_query}%"
            params.extend([search_param] * 4)
        
        if zip_param:
            base_query += " AND e.zip = ?"
            params.append(zip_param)
        
        if filter_type == 'upcoming':
            base_query += " AND e.date >= date('now') ORDER BY e.date ASC"
        elif filter_type == 'this_week':
            base_query += " AND e.date >= date('now') AND e.date <= date('now', '+7 days') ORDER BY e.date ASC"
        elif filter_type == 'by_provider':
            base_query += " ORDER BY COALESCE(p.business_name, p.first_name, 'Independent'), e.date ASC"
        else:
            base_query += " ORDER BY e.date DESC"
        
        cur.execute(base_query, params)
        events = cur.fetchall()
        
        cur.execute("SELECT DISTINCT zip FROM events ORDER BY zip")
        all_zips = [row[0] for row in cur.fetchall()]
        
        # Get profile for template
        cur.execute("SELECT first_name, business_name FROM profile WHERE id = 1")
        profile_row = cur.fetchone()
        profile = {
            "first_name": profile_row["first_name"] if profile_row else "",
            "business_name": profile_row["business_name"] if profile_row else "Your Service Provider"
        }
    
    return render_template(
        "events.html",
        events=events,
        all_zips=all_zips,
        profile=profile,
        search_query=search_query,
        filter_type=filter_type,
        zip_param=zip_param,
        title="Events"
    )